<a href="https://colab.research.google.com/github/Subhom-Ghosh/ML_Project/blob/main/Box_Office_Revenue_Prediction_Using_Linear_Regression_in_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import files
upload=files.upload()

Saving boxoffice.csv to boxoffice.csv


In [ ]:
df=pd.read_csv("boxoffice.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
to_remove=['world_revenue','opening_revenue']
df.drop(to_remove,axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
#checking missing values
df.isnull().sum()*100/df.shape[0]

In [ ]:
df.drop('budget',axis=1,inplace=True)
for col in ['MPAA','genres']:
  df[col]=df[col].fillna(df[col].mode()[0])
df.dropna(inplace=True)
df.isnull().sum().sum()

In [ ]:
df['domestic_revenue']=df['domestic_revenue'].astype('str').str[1:]
for col in ['domestic_revenue','opening_theaters','release_days']:
  df[col]=df[col].astype('str').str.replace(',','')
  temp=(~df[col].isnull())
  df[temp][col] = df[temp][col].convert_dtypes(float)

  df[col] = pd.to_numeric(df[col], errors='coerce')

In [ ]:
sb.countplot(df['MPAA'])

In [ ]:
df.groupby('MPAA')['domestic_revenue'].mean()

In [ ]:
plt.subplots(figsize=(15, 5))

features = ['domestic_revenue', 'opening_theaters', 'release_days']
for i, col in enumerate(features):
    plt.subplot(1, 3, i+1)
    sb.distplot(df[col])
plt.tight_layout()
plt.show()

In [ ]:
plt.subplots(figsize=(15, 5))
for i, col in enumerate(features):
    plt.subplot(1, 3, i+1)
    sb.boxplot(df[col])
plt.tight_layout()
plt.show()

In [ ]:
for col in features:
  df[col] = df[col].apply(lambda x: np.log10(x))

In [ ]:
plt.subplots(figsize=(15, 5))
for i, col in enumerate(features):
    plt.subplot(1, 3, i+1)
    sb.distplot(df[col])
plt.tight_layout()
plt.show()

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(df['genres'])
features = vectorizer.transform(df['genres']).toarray()

genres = vectorizer.get_feature_names_out()
for i, name in enumerate(genres):
	df[name] = features[:, i]

df.drop('genres', axis=1, inplace=True)

In [ ]:
removed = 0

if 'action' in df.columns and 'western' in df.columns:
    for col in df.loc[:, 'action':'western'].columns:

        if (df[col] == 0).mean() > 0.95:
            removed += 1
            df.drop(col, axis=1, inplace=True)

print(removed)
print(df.shape)

In [ ]:
for col in ['distributor', 'MPAA']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [ ]:
plt.figure(figsize=(8, 8))
sb.heatmap(df.select_dtypes(include=np.number).corr() > 0.8,
            annot=True,
            cbar=False)
plt.show()

In [ ]:
features = df.drop(['title', 'domestic_revenue'], axis=1)
target = df['domestic_revenue'].values

X_train, X_val, Y_train, Y_val = train_test_split(features, target,
									test_size=0.1,
									random_state=22)
X_train.shape, X_val.shape

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)